






### Instruction
Spam filtering is a beginner’s example of the document classification task which involves classifying an email as spam or non-spam (a.k.a. ham) mail. An email dataset will be provided. We will use the following steps to build this application:
1) Preparing the text data
2) Creating a word dictionary
3) Feature extraction
4) Training the classifier
5) Checking the results on the test set

### Preparing the text data
The data-set used here, is split into a training set and a test set containing 702 mails and 260 mails respectively, divided equally between spam and ham mails. You will easily recognize spam mails as it contains `spmsg` in its filename.

In any text mining problem, text cleaning is the first step where we remove those words from the document which may not contribute to the information we want to extract. Emails may contain a lot of undesirable characters like punctuation marks, stop words, digits, etc which may not be helpful in detecting the spam email. The emails in Ling-spam corpus have been already preprocessed in the following ways:

1. **Removal of stop words** – Stop words like “and”, “the”, “of”, etc are very common in all English sentences and are not very meaningful in deciding spam or legitimate status, so these words have been removed from the emails.

2. **Lemmatization** – It is the process of grouping together the different inflected forms of a word so they can be analyzed as a single item. For example, “include”, “includes,” and “included” would all be represented as “include”. The context of the sentence is also preserved in lemmatization as opposed to stemming (another buzz word in text mining which does not consider meaning of the sentence)

We still need to remove the non-words like punctuation marks or special characters from the mail documents. There are several ways to do it. Here, we will remove such words after creating a dictionary, which is a very convenient method to do so since when you have a dictionary; you need to remove every such word only once.

### Creating word dictionary
We will only perform text analytics on the content to detect the spam mails. As the first step, we need to create a dictionary of words and their frequency. For this task, a training set of 700 mails is utilized. This python function will create the dictionary for you.
```Python
def make_Dictionary(train_dir):
    emails = [os.path.join(train_dir,f) for f in os.listdir (train_dir)]
    all_words = []
    for mail in emails:
        with open (mail) as m:
            for i,line in enumerate (m) :
                if i == 2:
                    words = line.split()
                    all_words += words
    dictionary = Counter(all_words)
    # Paste code for non-word removal here
    
    return dictionary
```

Once the dictionary is created we can add just a few lines of code written below to the above function to remove non-words. Absurd single characters in the dictionary which are irrelevant here are also removed. Do not forget to insert the below code in the function of make_Dictionary:
```python
list_to_remove = list(dictionary.keys())
for item in list_to_remove:
    if item.isalpha() == False:
        del dictionary[item]
    elif len(item) == 1:
        del dictionary[item]
dictionary = dictionary.most_common(3000)
```

In [36]:
import os
import numpy as np
from collections import Counter
def make_Dictionary(train_dir):
    emails = [os.path.join(train_dir,f) for f in os.listdir (train_dir)]
    all_words = []
    for mail in emails:
        with open (mail) as m:
            for i,line in enumerate (m) :
                if i == 2:
                    words = line.split()
                    all_words += words
    dictionary = Counter(all_words)
    list_to_remove = list(dictionary.keys())
    for item in list_to_remove:
        if item.isalpha() == False:
            del dictionary[item]
        elif len(item) == 1:
            del dictionary[item]
    dictionary = dictionary.most_common(3000)
    # Paste code for non-word removal here
    return dictionary

The dictionary can be seen by the command “print dictionary”. You may find some absurd word counts to be high but don’t worry, it’s just a dictionary and you always have a chance to improve it later. If you use the provided dataset, make sure your dictionary has some of the entries given below as most frequent words. Here 3000 most frequently used words are chosen in the dictionary.

In [37]:
# To show the most frequent words in train-mails
dictionary = make_Dictionary('ling-spam/train-mails')
dictionary

[('order', 1414),
 ('address', 1293),
 ('report', 1216),
 ('mail', 1127),
 ('send', 1079),
 ('language', 1072),
 ('email', 1051),
 ('program', 1001),
 ('our', 987),
 ('list', 935),
 ('one', 917),
 ('name', 878),
 ('receive', 826),
 ('money', 788),
 ('free', 762),
 ('work', 755),
 ('information', 677),
 ('business', 654),
 ('please', 652),
 ('university', 595),
 ('us', 564),
 ('day', 556),
 ('follow', 544),
 ('internet', 520),
 ('over', 511),
 ('http', 479),
 ('check', 472),
 ('call', 469),
 ('each', 466),
 ('include', 452),
 ('com', 448),
 ('linguistic', 442),
 ('number', 423),
 ('want', 420),
 ('letter', 419),
 ('need', 418),
 ('many', 412),
 ('here', 397),
 ('market', 395),
 ('start', 390),
 ('even', 386),
 ('fax', 383),
 ('form', 380),
 ('most', 377),
 ('first', 373),
 ('web', 366),
 ('service', 363),
 ('interest', 362),
 ('software', 352),
 ('remove', 349),
 ('read', 347),
 ('those', 345),
 ('week', 344),
 ('every', 332),
 ('credit', 329),
 ('ll', 326),
 ('site', 320),
 ('much', 31

### Feature Extraction Process
Once the dictionary is ready, we can extract word count vector (our feature here) of 3000 dimensions for each email of the training set. Each **word count vector** contains the frequency of 3000 words in the training file. Of course you might have guessed by now that most of them will be zero. Let us take an example. Suppose we have 500 words in our dictionary. Each word count vector contains the frequency of 500 dictionary words in the training file. Suppose the text in the training file is “Get the work done, work done”, then it will be encoded as $$[0,0,0,0,0,…….0,0,2,0,0,0,……,0,0,1,0,0,…0,0,1,0,0,……2,0,0,0,0,0]$$ Here, all the word counts are placed at the 296th, 359th, 415th, 495th elements of the word count vector in the length of 500 and the rest are zero.

The below python code will generate a feature vector matrix whose rows denote 700 files of the training set and columns denote 3000 words of the dictionary. The value at index ${ij}$ will be the number of occurrences of the $j^{th}$ word of the dictionary in the $i^{th}$ file

In [38]:
def extract_features(mail_dir):
    files = [os.path.join(mail_dir,fi) for fi in os.listdir(mail_dir)]
    features_matrix = np.zeros((len(files),3000))
    label = np.zeros(len(files))
    docID = 0
    _i = 0
    print(len(files))
    for fil in files:
        _i+=1
        label[docID] = int('spmsg' in fil)
        with open(fil) as fi:
            for i,line in enumerate(fi):
                if i == 2:
                    words = line.split()
                    for word in words:
                        wordID = 0
                        for i,d in enumerate(dictionary):
                            if d[0] == word:
                                wordID = i
                                features_matrix[docID,wordID]+=1
            docID = docID + 1
        print('\r','done {} files'.format(_i),flush=True,end='')
    return features_matrix, label

### Training the Classifiers

In [39]:
x_train, y_train = extract_features("ling-spam/train-mails")
x_test, y_test = extract_features("ling-spam/test-mails")

702
 done 702 files260
 done 260 files

In [40]:
class GaussianNB:
    def __init__(self):
        self.classes = None
        self.mean = {}
        self.var = {}
        self.prior = {}

    def fit(self, X, y):
        self.classes = np.unique(y)
        for _, c in enumerate(self.classes):
            label = "class" + str(c)
            X_Index_c = X[np.where(y == c)]
            self.mean[label] = np.mean(X_Index_c, axis=0, keepdims=True)
            self.var[label] = np.var(X_Index_c, axis=0, keepdims=True)
            self.prior[label] = X_Index_c.shape[0] / X.shape[0]

    def predict(self, X):
        # return class with the highest probability
        prediction = []
        for cur_X in X:
            cur_result = {}
            for c in self.classes:
                label = "class" + str(c)
                mean = self.mean[label][0]
                var = self.var[label][0]
                prior = np.log(self.prior[label])
                for x, m, v in zip(cur_X, mean, var):
                    if v:
                        x_pro = (
                            1.0
                            / np.sqrt(2.0 * np.pi * v)
                            * np.exp(-np.square(x - m) / (2.0 * v))
                        )
                    else:
                        x_pro = int(m == x)
                    prior += np.log(x_pro + 1e-200)
                cur_result[c] = prior
            prediction.append(max(cur_result, key=cur_result.get))
        return prediction

### Checking Performance
The test set contains 130 spam emails and 130 non-spam emails. Please compute accuracy, recall, F-1 score to evaluate the performance of your spam filter.

In [41]:
############################################
from sklearn.metrics import accuracy_score, classification_report

model = GaussianNB()
model.fit(x_train, y_train)
y_pred = model.predict(x_test)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Custom Accuracy:", accuracy)

# Generate a classification report
report = classification_report(y_test, y_pred)
print("Custom Classification Report:\n", report)
############################################
from sklearn.naive_bayes import GaussianNB as SKGaussianNB
model = SKGaussianNB()
model.fit(x_train, y_train)
y_pred = model.predict(x_test)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Sklearn Accuracy:", accuracy)

# Generate a classification report
report = classification_report(y_test, y_pred)
print("Sklearn Classification Report:\n", report)
############################################

Custom Accuracy: 0.9730769230769231
Custom Classification Report:
               precision    recall  f1-score   support

         0.0       0.95      1.00      0.97       130
         1.0       1.00      0.95      0.97       130

    accuracy                           0.97       260
   macro avg       0.97      0.97      0.97       260
weighted avg       0.97      0.97      0.97       260

Sklearn Accuracy: 0.9615384615384616
Sklearn Classification Report:
               precision    recall  f1-score   support

         0.0       0.93      0.99      0.96       130
         1.0       0.99      0.93      0.96       130

    accuracy                           0.96       260
   macro avg       0.96      0.96      0.96       260
weighted avg       0.96      0.96      0.96       260



### Questions
1. Describe another real-world application where the naïve Bayes method can be applied
2. What are the strengths of the naïve Bayes method; when does it perform well?
3. What are the weaknesses of the naïve Bayes method; when does it perform poorly?
4. What makes the naïve Bayes method a good candidate for the classification problem, if you have enough knowledge about the data?

1. **Real-World Application of Naive Bayes:**
   Naive Bayes is a versatile algorithm used in various real-world applications. One such application is **sentiment analysis** in natural language processing. 

2. **Strengths of Naive Bayes:**
   - **Simplicity:** Naive Bayes is a straightforward and easy-to-understand algorithm, making it suitable for both beginners and experts.
   - **Efficiency:** It is computationally efficient and works well with high-dimensional data.
   - **Good for Text Data:** Naive Bayes often performs well on text classification tasks, such as spam detection and sentiment analysis.
   - **Handles Missing Data:** It can handle missing values gracefully by simply ignoring them.
   - **Scalability:** Naive Bayes is highly scalable, making it suitable for large datasets.

   It performs well when:
   - Features are conditionally independent (the "naive" assumption).
   - The dataset has categorical or discrete features.
   - There are many features relative to the size of the dataset.

3. **Weaknesses of Naive Bayes:**
   - **Simplistic Assumption:** The primary weakness is the assumption of feature independence, which may not hold true in many real-world datasets.
   - **Limited Expressiveness:** Naive Bayes may not capture complex relationships between features.
   - **Sensitive to Data Quality:** It can be sensitive to the quality of training data, especially when dealing with imbalanced datasets.
   - **Doesn't Handle Continuous Data Well:** It's not the best choice for datasets with continuous features unless discretization is applied.

   It performs poorly when:
   - Features are not conditionally independent (violation of the naive assumption).
   - The dataset contains a significant amount of continuous or real-valued data.
   - The data quality is poor, or there's a class imbalance issue.

4. **Why Naive Bayes for Classification:**
   Naive Bayes can be a good candidate for classification problems, especially when you have enough knowledge about the data, because:
   - It's computationally efficient and scales well, even with large datasets.
   - It's easy to implement and interpret, making it a quick choice for prototyping and initial analysis.
   - It often performs surprisingly well on text data, making it suitable for tasks like spam detection, sentiment analysis, and document classification.
   - It's a useful baseline model for text classification tasks, providing a simple benchmark against which more complex models can be compared.